In [12]:
import pandas as pd
import torch
import helpful
import PIL

In [2]:
df = pd.read_csv("../data/imagenette/validation_labels.csv")

In [3]:
df

,id,label,split
0,val/n02979186/n02979186_8971.JPEG,cassette player,valid
1,val/n02979186/n02979186_14550.JPEG,cassette player,valid
2,val/n02979186/n02979186_11971.JPEG,cassette player,valid
3,val/n02979186/n02979186_11550.JPEG,cassette player,valid
4,val/n02979186/n02979186_8751.JPEG,cassette player,valid
...,...,...,...
3920,val/n03425413/n03425413_17521.JPEG,gas pump,valid
3921,val/n03425413/n03425413_20711.JPEG,gas pump,valid
3922,val/n03425413/n03425413_19050.JPEG,gas pump,valid
3923,val/n03425413/n03425413_13831.JPEG,gas pump,valid


In [5]:
model, preprocess, tokenizer = helpful.load_clip()

In [23]:
# given a single text string, return the score for the best matches for the images
with torch.no_grad(), torch.cuda.amp.autocast():
	my_text = "a picture of a radio"
	# [1, 77]
	my_text_tensor = tokenizer([my_text])

	# [1, 512]
	my_text_features = model.encode_text(my_text_tensor)

	# now encode all the images into a single gigatensor
	
	# TODO don't do these separate
	def image_path_to_PIL(df: pd.Series):
		return df.apply(lambda x: PIL.Image.open(f"../data/imagenette/{x}"))
	def preprocess_PIL(df: pd.Series):
		return df.apply(lambda x: preprocess(x).unsqueeze(0))
	
	image_col = df["id"]
	pil_col = image_path_to_PIL(image_col)
	list_of_image_tensors = preprocess_PIL(pil_col).tolist()

	# [3925, 3, 224, 224]
	image_tensor = torch.cat(list_of_image_tensors[0:2], dim=0)

	# [3925, 512]
	image_features = model.encode_image(image_tensor)
	print(image_features.shape)

	# [3925, 1]
	clip_scores = image_features @ my_text_features.T

torch.Size([2, 512])
